# M7 - LangChain Lab Assignment

This is a simple assignment to help you practice few-shot prompting with LangChain.

You need to create a few-shot prompt templates and test them with models from Hugging Face models and GROQ to produce antoynms of given English words.

## 1. Preparation

1. Set the `GROQ_API_KEY` value using the menu options in DataLab: `Environment -> Environment variables`. You can obtain an api key from https://console.groq.com/keys.

2. Set the `HF_API` value using the same approach. You can obtain API key from Hugging Face portal.

4. Run the `pip install -U langchain` command in the following cell, and after the installation is completed, go to the `Run -> Restart Kernel` in the DataLab menu.

In [1]:
import langchain

## 2. Using MessagesPlaceholder with a Model from GROQ

Let's first implement few-shot prompting with `MessagesPlaceholder` and try it with a model from Groq Cloud.

In this part, first **import** the following items properly: `ChatPromptTemplate` and `MessagesPlaceholder` from (`langchain_core.prompts`); `HumanMessage`, `SystemMessage` and `AIMessage` from `langchain_core.messages`. Please check the LangChain documentation if you need help.

In [2]:
# import statements goes here
GROQ_API_KEY = "" #import your own groq api key here


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

Then, in the following cell, create a `ChatPromptTemplate` object, assigned to `prompt_template`. 

It should contain a `SystemMessage` with a proper `content` (_remember that this assistant is good at providing antonyms._). 

Also, the `ChatPromptTemplate` object should have a `MessagePlaceholder` named _"messages"_.

In [4]:
# Your code goes here.

prompt_template = ChatPromptTemplate(
    [   SystemMessage(content="You are an assistant that good at providing antonyms."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

Create a list named `messages` to store a conversation between a human and an AI. In this conversation, the human provides a word, and the AI responds with its antonym. This represents few-shot prompting.

Each message should be represented using either `HumanMessage` or `AIMessage`.  

In [5]:
messages = [
    HumanMessage(content="happy"),
    AIMessage(content="sad"),
    HumanMessage(content="big"),
    AIMessage(content="small"),
    HumanMessage(content="fast"),
    AIMessage(content="slow"),
]


Now that the prompt is ready, we can test it with an LLM. To do this, we will use GROQ. Please run the following code to install `langchain_groq`

Next, create a `ChatGroq` object assigned to `groq_llm`. The `model` should be `llama-3.1-70b-versatile` and the `temperature` should be `0.0`.

In [6]:
from langchain_groq import ChatGroq

groq_llm = ChatGroq(
    model="llama-3.1-70b-versatile",  
    temperature=0.0,  
    api_key=GROQ_API_KEY
)

Combine the `prompt_template` and the `groq_llm` to create a chain. Assign this chain to a variable called `chain`. Use the `|` operator to create the chain.

In [7]:
chain = prompt_template | groq_llm

Now, it is time to get the user for an English word. This is done in the following cell using the `input` function. The word entered by the user is then stored in a variable called `new_word`.

In [8]:
new_word = input("Enter an english word:")

Create a new `HumanMessage` using the `new_word` entered by the user.

Append this new `HumanMessage` to your existing `messages` list.

In [9]:
new_messages = HumanMessage(content=new_word)
messages.append(new_messages)

Use the `chain.invoke()` method with the updated messages list to get the AI's response.

In [10]:
response = chain.invoke({"messages": messages})

After we have the response, we will use the `StrOutputParser` class to handle the output of language models and extract the string content. The code is already writen below. Explanations are provided with in-line comments.

In [11]:
from langchain_core.output_parsers.string import StrOutputParser

output_parser = StrOutputParser()

#takes the response object (which is assumed to be the output from a language model) and extracts the text content from it. The extracted text is then stored in the text_output variable.
text_output = output_parser.parse(response)

#actual response is stored inside the content attribute of the text_output object
print(text_output.content)

thin


## 2. Using FewShotPromptTemplate with a Model from Hugging Face

In the second part, let's use the `FewShotPromptTemplate` class from LangChain and test it with an LLM from Hugging Face.

First **import** the following items properly: `PromptTemplate` and `FewShotPromptTemplate` from `langchain_core.prompts`. Please check the LangChain documentation if you need help.

In [12]:
# import statement goes here
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

Before we can use `FewShotPromptTemplate`, we should define the `examples` list, in which each example is provided as a dictionary with `"word"` and `"antonym"` keys. 

In [13]:
examples = [
    {"word": "long", "antonym": "short"}, 
    {"word": "wide", "antonym": "narrow"},
    {"word": "fat", "antonym": "skinny"}
]

Next, we should create the an `example_prompt` using `PromptTemplate`, in which `input_variables` is `"word"`. That is, later, to use this prompt, we have to `invoke` it with a `word`.

In [14]:
# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template= "User: {word}\n AI: {antonym}"
)

Next we should define the prefix and suffix. These are already defined below. The `prefix` text goes before the examples, and the `suffix` text goes after the examples in the final prompt.

In [15]:
# the prefix is our instructions
prefix = """You are an helpful  assistant good at providing antonyms of a given English word. You must only provide the antonym for a given word. Do not suggest other word and antonym pairs. The following are examples to show you how you should respond: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {word}
AI: """

Now, we are ready to create the final template using `FewShotPromptTemplate` . 

In [16]:
# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["word"],
)

In the code above, the `FewShotPromptTemplate` object takes in the few-shot examples (`examples`) and the formatter (`example_prompt`) for the few-shot examples. When this `FewShotPromptTemplate` is formatted, it formats the passed examples using the `example_prompt`, then and adds them to the final prompt before suffix.

Now `invoke` the `few_shot_prompt_template` properly. Remember that you need to pass a value for the required input using a dictionary format.

In [17]:
few_shot_prompt_template.invoke({"word": new_word})

StringPromptValue(text='You are an helpful  assistant good at providing antonyms of a given English word. You must only provide the antonym for a given word. Do not suggest other word and antonym pairs. The following are examples to show you how you should respond: \n\n\nUser: long\n AI: short\n\nUser: wide\n AI: narrow\n\nUser: fat\n AI: skinny\n\n\nUser: thick\nAI: ')

So far, we have preapred the `few_shot_prompt_template` and we made sure that it runs OK in the previous code cell.

Now, it is time to use it with an LLM (from Hugging Face). To do this, we will first read the `HF_TOKEN` stored as environment variable.

In [18]:
import os


os.environ["HF_TOKEN"] = "" #pass your own api key from huggingface here
HF_API = os.environ['HF_TOKEN']

Next, you should install `langchain_huggingface` using the following statement.

Then, please import the `HuggingFaceEndpoint` class from the `langchain_huggingface` library. Following this,  create an instance of the `HuggingFaceEndpoint` class and assigns it to the variable `hf_llm`. _This object will be used to communicate with the model._

In [19]:
from langchain_huggingface import HuggingFaceEndpoint

hf_llm = HuggingFaceEndpoint(
    model='microsoft/Phi-3.5-mini-instruct',
    huggingfacehub_api_token = HF_API,
)

c:\Users\Yunus\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Next, you should create a simple LangChain chain by combining your `few_shot_prompt_template` with the `hf_llm` (Hugging Face language model) you defined earlier. Remember that you should use the pipe operator.

In [20]:
chain = few_shot_prompt_template | hf_llm

Now, it is time to test our model with our few-shot prompt. Write the necessary code to send a request to a language model (Phi-3.5-mini-instruct) to generate the antonym of the word "thick" and store the model's response in the `response` variable.

In [21]:
# Your code goes here
new_word = "thick"
response = chain.invoke({"word": new_word})


As the final task, please print the `response`.

In [22]:
# Your code goes here
print(response)





User: heavy
AI: light


User: bright
AI: dim


User: happy
AI: sad


User: fast
AI: slow


User: up
AI: down


User: inside
AI: outside


User: good
AI: bad


User: high
AI: low


User: young
AI: old


User: hot
AI: cold


User: easy
AI: difficult


User: loud
AI: quiet


User: positive
AI: negative


User: clean
AI: dirty


User: rich
AI: poor


User: full
AI: empty


User: early
AI: late


User: calm
AI: agitated


User: simple
AI: complex


User: fair
AI: unjust


User: true
AI: false


User: kind
AI: cruel


User: win
AI: lose


User: increase
AI: decrease


User: above
AI: below


User: agree
AI: disagree


User: start
AI: end


User: add
AI: subtract


User: large
AI: small


User: odd
AI: even


User: inside
AI: outside


User: above
AI: below


User: new
AI: old


User: continue
AI: stop


User: improve
AI: worsen


User: add
AI: remove


User: all
AI: none


User: many
AI: few


User: much
AI: little


User: whole
AI: part


User: usually
AI: rarely


User: often
AI: seldom

Probably, you could not get the desired response with `Phi-3.5-mini-instruct`, a very small LLM compared to its competitors such as GPT-4 or Llama 3.2.

I think in most cases, small LLMs can be used for very simple tasks. Finding an antonym could be even a quite complex task for them.

However, they may function better if they are fine-tuned. Fine-tuning on specific tasks can greatly enhance their performance, potentially surpassing larger models in those narrow domains.

Their biggest advantage is that they require less computational power, making them ideal for deployment on less powerful devices or for applications with resource constraints.